<img src='pics/otus.png'>

In [5]:
from __future__ import print_function
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib
import pandas_profiling
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [8]:
fn = 'orders.csv'

In [9]:
df = pd.read_csv(fn)
df.head(5)

,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
0,23.55%,Хроники Амбера. Том 2,357.00,1,2017-03-19 22:25:20,110.00,169108,602051,1,527.00,911531
1,4.07%,Хроники Амбера. Том 1,448.00,1,2017-03-19 22:19:58,19.00,1394744,602047,1,618.00,911531
2,4.17%,Пищеводитель,414.00,1,2017-03-19 20:58:00,65.00,191276,602017,4,1503.00,911587
3,4.17%,Аудиокн. Мясников. &quot;Ржавчина&quot;: что д...,253.00,1,2017-03-19 20:58:00,65.00,157945,602017,4,1503.00,911587
4,4.09%,Инфекции. Как защитить себя и своего ребенка,422.00,1,2017-03-19 20:58:00,65.00,205173,602017,4,1503.00,911587


# 1. Провести EDA
Просмотреть структуру данных, значения каждого признака, поискать особенности в признаках и в данных вцелом

In [33]:
"""
формула рассчета изначальной цены за книгу (без скидки)
"""
def book_price(price, discount):
    #price = nominal_price - (nominal_price/100) * discount
    #price = nominal_price * (1 - discount/100)
    nominal_price = price/(1 - discount/100)
    return round(nominal_price)
    

In [34]:
print(book_price(263,20.06))
print(book_price(316,3.95))

329
329


In [12]:
print(df.count())
df.describe()
df.sort_values(['NAME', 'date_insert']).head(5)

DISCOUNT_VALUE        553777
NAME                  557811
PRICE                 557818
QUANTITY              557818
date_insert           557818
discount              557818
itemID                557818
orderID               557818
ordered_item_count    557818
price                 557818
user_id               557818
dtype: int64


,DISCOUNT_VALUE,NAME,PRICE,QUANTITY,date_insert,discount,itemID,orderID,ordered_item_count,price,user_id
60033,20.06%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",263.00,1,2017-03-31 15:54:33,133.00,1418863,610585,4,1138.00,917439
63068,3.95%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",316.00,1,2017-04-04 17:32:24,256.00,1418863,612159,5,1285.00,220005
128639,23.1%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",253.00,1,2017-05-02 23:22:57,2066.00,1418863,626643,32,7144.00,38770
130829,23.1%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",253.00,2,2017-05-02 23:27:03,604.00,1418863,626649,9,5349.00,38770
135324,10.03%,"# Мужчины с Марса, женщины с Венеры. Курс испо...",329.00,1,2017-05-11 18:44:04,187.00,1418863,630919,4,1058.00,931194


In [48]:
print(df[df['NAME'].isnull()])
#df.groupby(['NAME']).agg({'QUANTITY': 'sum', 'PRICE': 'sum'}).sort_values('QUANTITY', ascending=False).head(100)
#df['NAME'] = df['NAME'].fillna('-')
#df[df.NAME.str.match(r'[a-zA-Z]+')]

Empty DataFrame
Columns: [DISCOUNT_VALUE, NAME, PRICE, QUANTITY, date_insert, discount, itemID, orderID, ordered_item_count, price, user_id, BOOK_PRICE]
Index: []


1. нужно NaN значения DISCOUNT_VALUE привести к 0
2. NaN  значения NAME удалить 
3. DISCOUNT_VALUE  привести к цифровому значению
4. Добавить колонку book_price  где будет цена книги без скидки

# 2. Спланировать эксперимент 

Задача:
* сегментация товаров + сегментация пользователей (по товарам, по поведению, скидкам и пр.)

Дополнительнительные задачи (опционально, подумайте о них, могут появиться интересные идеи и находки):
* поиск товаров-заменителей и товаров, дополняющих друг друга
* поиск выбросов среди товаров
* поиск выбросов среди заказов

Эксперимент - составить план, по которому будет проделана исследовательская работа. Примерные пункты плана:
* Составить первичный список признаков, необходимых для решения задачи
* Сгенерировать список новых признаков, которые можно получить из данных
* Применить понижение размерности для визуализации данных
* Применить кластеризацию, визуализировать
* Описание кластера
* Любые другие пункты

# 3. Выполнить подготовку данных для построения модели

Выполнить необходимые преобразования данных

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class TransformPercent(BaseEstimator, TransformerMixin):
    
    def __init__(self, *args):
        self._field_name = args[0]
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df,  y=None):
        _df = df.copy()  # делаем копию данных (полагая, что df - датафрейм pandas),   
        _df[self._field_name] = _df[self._field_name].replace('[\%]','',regex=True).astype(float)
        _df[self._field_name].fillna(0.0)
        return _df    
    

In [27]:
model_pipe = Pipeline([('percent', TransformPercent('DISCOUNT_VALUE'))])
model_pipe.fit(df)
analyze_df = model_pipe.transform(df)

In [35]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, Imputer, LabelEncoder

def get_discount_num_and_price(df):
    df['DISCOUNT_VALUE'] = df['DISCOUNT_VALUE'].replace('[\%]','',regex=True).astype(float)
    df['DISCOUNT_VALUE'].fillna(0.0)
    df['BOOK_PRICE'] = df[['PRICE','DISCOUNT_VALUE']].apply(lambda x: book_price(*x), axis=1)
    return df[['DISCOUNT_VALUE', 'BOOK_PRICE']]    

def get_name_col(df):
    return df['NAME']

#pipeline = make_union(*[
#    make_pipeline(FunctionTransformer(get_discount_num_and_price, validate=False), Imputer(strategy='mean'), StandardScaler()),
#])
get_discount_num_and_price(df)                          

,DISCOUNT_VALUE,BOOK_PRICE
0,23.55,467.00
1,4.07,467.00
2,4.17,432.00
3,4.17,264.00
4,4.09,440.00
5,4.17,432.00
6,4.12,510.00
7,4.12,510.00
8,4.09,611.00
9,4.49,178.00


# 4. Провести построение модели 

# 5. Проанализировать результат
получилось или нет, почему, что можно улучшить, можно ли интерпретировать результат

# 6. Повторить при необходимости 

# Подсказки

* Проводить EDA и работу с признаками на всем датасете
* Для построения модели отсеивать данные тем или иным способом (обосновать выбор) - уменьшить объем данных для анализа
* Оформлять для сдачи задания все эксперименты (в том числе неудачные). Так мы сможем увидеть процесс работы над заданием
* Не тратить 90% времени на первый этап. Планировать время так, чтобы успеть поработать над моделью (в равной степени над каждым пунктом).